In [1]:
import pandas as pd, numpy as np

## get FIPS codes with county names

In [22]:
df = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt', header=None)
#convert to strings, pad with zeros, 2 digits for state, 3 for county
df[1] = df[1].map(lambda x: str(x))
df[2] = df[2].map(lambda x: str(x)).apply(lambda x: x.zfill(3))

#add FIPS or id column and test
df['id'] = df[1] + df[2]
df['name'] = df[3].str.replace(' County','')
fips = df[['id','name']]

## Load data, sum over years, and merge

In [48]:
df = pd.read_csv('http://cdiac.ess-dive.lbl.gov/carbonmanagement/soilcarbonchange/1990_2008soilcarbonchange_US_v2.csv')
col_list = list(df)
col_list.remove('FIPS') #because it's an int64 type
df['change'] = df[col_list].sum(axis=1)
df['change'] = df['change']/1000 #thousand tons
df['change'].describe()


count    3109.000000
mean     -100.044105
std       156.515993
min     -1914.829730
25%      -134.653341
50%       -36.333978
75%        -5.814587
max         0.431752
Name: change, dtype: float64

In [50]:
df['id'] = df['FIPS'].apply(str)
change = pd.merge(fips, df, on='id', how='inner')
change = change[['id','name','change']]
#id or FIPS code needs to be 5-digit string to work with d3 topojson
change['id']= change['id'].map(lambda x: str(x)).apply(lambda x: x.zfill(5))
change['change'] = change['change'].round(2)
change.to_csv('estimated_C_change_by_county_1990-2008.tsv', sep='\t',index=False)